Importing Libraries and Load Dataset

In [4]:
import pandas as pd #to read the file and manipulate data
import numpy as np #powerful libary to do mathematical equation
import re #regular expression to filter strings
import polars as pl

df_games = pd.read_csv('steam_games.csv') #
df_games.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'steam_games.csv'

In [ ]:
# Read each file into a dataframe and store them in a list
dfs = [pl.read_parquet("data/parquets/2066020_reviews_8125.parquet")]

# Concatenate all dataframes into one
df = pl.concat(dfs)

In [ ]:
import glob
import polars as pl

# Get a list of all .parquet files in the directory
files = glob.glob('data/parquets/*.parquet')

# Read each file into a dataframe and store them in a list
dfs = [pl.read_parquet(file) for file in files]

# Concatenate all dataframes into one
df = pl.concat(dfs)

In [5]:
# Data understanding
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121904 entries, 0 to 121903
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   title             121893 non-null  object 
 1   url               121904 non-null  object 
 2   image             121904 non-null  object 
 3   release_date      113850 non-null  object 
 4   platforms         117576 non-null  object 
 5   discount_rate     2941 non-null    object 
 6   original_price    109675 non-null  object 
 7   discounted_price  2935 non-null    object 
 8   developer         113921 non-null  object 
 9   publisher         103890 non-null  object 
 10  overall_reviews   78125 non-null   object 
 11  recent_reviews    78125 non-null   object 
 12  whole_reviews     5773 non-null    object 
 13  description       75925 non-null   object 
 14  tags              112020 non-null  object 
 15  genre             111334 non-null  object 
 16  processor         10

In [6]:
# Check for duplicates

df_games.duplicated().any() # This will return boolean value

# df_games.drop_duplicates(inplace=True) # use this if any dups 

False

In [8]:
# Removing Unused Columns

df_games_drop = df_games.copy() # create a copy of our data
drop = ['url','image','discount_rate','discounted_price','description','metacriticts','processor','graphic_card'] #add/remove columns here
df_games_drop = df_games_drop.drop(labels=drop,axis=1)

In [9]:
# Data Extraction and Formatting

def extract_year(x): #function to extract year from list of numbers
    year = None
    if type(x) == list:
        for i in x:
            if len(i) == 4:
                year = i
                return year
            else: 
                year = None
        return year
    else:
        year = x
    return year
df_games_extract = df_games_drop.copy()
df_games_extract['year'] = df_games_extract['release_date'].str.findall('(\d+)').apply(lambda x: extract_year(x)).astype(float)

In [10]:
df_games_drop.sample(5)

,title,release_date,platforms,original_price,developer,publisher,overall_reviews,recent_reviews,whole_reviews,tags,genre,ram,rating,language
7775,Ruined King: A League of Legends Story™ - Ruin...,"16 Nov, 2021",Windows,Rp 54 999,Airship Syndicate,Riot Forge,Mostly Positive,- 78% of the 56 user reviews for this game are...,NaN,"RPG,Indie","Indie,RPG",8 GB RAM,NaN,"English,French,Italian,German,Spanish - Spain,..."
53349,accelerating hotkeys,"28 Jan, 2022","Windows,Mac OS,Linux",Rp 39 999,Rising Sun Interactive,Rising Sun Interactive,1 user reviews,- Need more user reviews to generate a score,NaN,"Typing,Time Management,Immersive Sim,Abstract,...","Action,Indie,Simulation",2 GB RAM,NaN,"English,German,French,Italian,Spanish - Spain"
35180,Coloring Game 4 – Insta,"30 Jul, 2021",Windows,Rp 17 499,L. Stotch,L. Stotch,Mostly Negative,- 30% of the 10 user reviews for this game are...,NaN,"Free to Play,Indie,Casual,Relaxing,Great Sound...","Casual,Free to Play,Indie",1024 MB RAM,NaN,"English,French,Italian,German,Spanish - Spain,..."
8812,Bridge Constructor Medieval,"15 Sep, 2014","Windows,Mac OS,Linux",Rp 89 999,ClockStone,Headup Games,Very Positive,- 81% of the 188 user reviews for this game ar...,NaN,"Simulation,Casual,Indie,Building,Puzzle,Mediev...","Casual,Indie,Simulation",1 GB RAM,NaN,"English,French,Italian,German,Spanish - Spain,..."
101131,Visual Novel Maker - Kingdoms and Conquest Mus...,"12 Dec, 2019","Windows,Mac OS,Linux",Rp 95 999,Joel Steudler,Degica,NaN,NaN,NaN,Web Publishing,Web Publishing,NaN,NaN,"English,Japanese"


In [16]:
# extracting the month and year of game release

def extract_year(x): #function to extract year from list of numbers
    year = None
    if type(x) == list:
        for i in x:
            if len(i) == 4:
                year = i
                return year
            else: 
                year = None
        return year
    else:
        year = x
    return year

In [17]:
#df_games_drop[['date','month','year']] = df_games_drop['release_date'].str.split(' ', n=2 , expand=True)
df_games_extract = df_games_drop.copy()
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#df_games_extract['year'] = df_games_extract['release_date'].str[-4:].apply(pd.to_numeric, errors=('coerce')) 
df_games_extract['year'] = df_games_extract['release_date'].str.findall('(\d+)').apply(lambda x: extract_year(x)).astype(float)
df_games_extract['month'] = df_games_extract['release_date'].str.findall("[a-zA-Z]+").str[0]
df_games_extract['month'] = df_games_extract['month'].replace(['August','June'],['Aug','Jun'])
df_games_extract.loc[~df_games_extract['month'].isin(month),'month'] = None
df_games_extract = df_games_extract.drop('release_date', axis=1) #drop release date
df_games_extract.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,recent_reviews,whole_reviews,tags,genre,ram,rating,language,year,month
68230,RealmLess,Windows,NaN,ChuYou.Ltd,WeNeedPublisher,NaN,NaN,NaN,"Action,Adventure,Action-Adventure,Action Rogue...","Action,Adventure,Indie",4 GB RAM,NaN,"English,Simplified Chinese",2022.0,None
55256,Fantasy Dungeon,Windows,Rp 24 999,Luci Entertainment,Luci Entertainment,5 user reviews,- Need more user reviews to generate a score,NaN,"Action Roguelike,Action,Casual,Shooter,Action ...","Action,Casual,Indie",512 MB RAM,NaN,English,2021.0,Apr
121536,Fantasy Grounds Unity,"Windows,Mac OS,Linux",Rp 169 999,"SmiteWorks USA, LLC","SmiteWorks USA, LLC",Very Positive,- 81% of the 11 user reviews in the last 30 da...,- 86% of the 376 user reviews for this game ar...,"RPG,Dungeons & Dragons,Strategy,Indie,Utilitie...","Indie,RPG,Strategy",4 GB RAM,NaN,English,2020.0,Nov
59493,RoboPhobik Soundtrack,NaN,Rp 32 999,Haruneko Entertainment,Haruneko Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0,Sep
72293,Panta Rhei,"Windows,Mac OS",NaN,Lone Grove,Lone Grove,NaN,NaN,NaN,"Action,Adventure,Action-Adventure,Roguelike,Ac...","Action,Adventure,Indie",6 GB RAM,NaN,"English,German",NaN,None


In [18]:
# Extract user ratings and total reviews
df_games_extract_2 = df_games_extract.copy()
df_games_extract_2['whole_reviews'] = df_games_extract_2['whole_reviews'].str.replace(',','')
df_games_extract_2['user_rating_all'] = df_games_extract_2['whole_reviews'].str.findall('(\d+)').str[0].astype(float)
df_games_extract_2['total_review_all'] = df_games_extract_2['whole_reviews'].str.findall('(\d+)').str[1].astype(float)
df_games_extract_2 = df_games_extract_2.drop('whole_reviews',axis=1)
df_games_extract_2.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,recent_reviews,tags,genre,ram,rating,language,year,month,user_rating_all,total_review_all
45771,Heart's Medicine - Time to Heal - Soundtrack,"Windows,Mac OS",Rp 24 999,Blue Giraffe,GameHouse,7 user reviews,- Need more user reviews to generate a score,"Adventure,Indie,Casual","Adventure,Casual,Indie",512 MB RAM,NaN,"English,French,Italian,German,Spanish - Spain,...",2016.0,Sep,NaN,NaN
95217,Kill Spree®,Windows,Free,Wunzero,Wunzero,NaN,NaN,"First-Person,Combat,Sports,Gore,Physics,Parkou...","Action,Adventure,Free to Play,Massively Multip...",16 GB RAM,NaN,English,2024.0,None,NaN,NaN
69491,Tiger Tank 59 Ⅰ Winter Assault MP073,Windows,Rp 8 499,TigerQiuQiu,TigerQiuQiu,NaN,NaN,"Action,Indie,Casual,Old School,Pixel Graphics,...","Action,Casual,Indie",4 GB RAM,NaN,English,2021.0,Aug,NaN,NaN
51460,Wild Terra Online - Lord Pack,"Windows,Mac OS,Linux",Rp 108 999,Juvty Worlds Ltd.,NaN,1 user reviews,- Need more user reviews to generate a score,"Action,Adventure,RPG,Massively Multiplayer,Indie","Action,Adventure,Indie,Massively Multiplayer,RPG",4 GB RAM,NaN,"English,German,Spanish - Spain,Bulgarian,Polis...",2019.0,Feb,NaN,NaN
116010,Bad Pad,"Windows,Linux",Rp 82 999,"Headbang Games,Oray Studios",Headbang Games,Positive,- 84% of the 26 user reviews for this game are...,"Metroidvania,Precision Platformer,Side Scrolle...","Action,Indie",2 GB RAM,NaN,"English,French,German,Spanish - Spain,Russian,...",2018.0,Aug,NaN,NaN


In [19]:
# extracting recent reviews

df_games_extract_2['recent_reviews'] = df_games_extract_2['recent_reviews'].str.replace(',','')
df_games_extract_2['user_rating_recent'] = df_games_extract_2['recent_reviews'].str.findall('(\d+)').str[0].astype(float)
df_games_extract_2['total_review_recent'] = df_games_extract_2['recent_reviews'].str.findall('(\d+)').str[1].astype(float)
df_games_extract_2 = df_games_extract_2.drop('recent_reviews',axis=1)
df_games_extract_2.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,tags,genre,ram,rating,language,year,month,user_rating_all,total_review_all,user_rating_recent,total_review_recent
44835,Jam Studio VR,Windows,Rp 108 999,"Beamz Interactive, Inc.","Beamz Interactive, Inc.,Vive Studios",6 user reviews,"Casual,Music,VR",Casual,4 GB RAM,NaN,"English,Simplified Chinese",2017.0,Sep,NaN,NaN,NaN,NaN
28177,Caffeine,Windows,Free,"Caffeine, Inc","Caffeine, Inc",Positive,"Utilities,Audio Production,Early Access,Video ...","Audio Production,Utilities,Video Production,Ea...",4 GB RAM,NaN,English,2018.0,May,NaN,NaN,83.0,31.0
94868,BIT.TRIP.FATE Soundtrack,"Windows,Mac OS",Rp 55 999,Gaijin Games,Gaijin Games,3 user reviews,"Action,Indie,Soundtrack","Action,Indie",512 MB RAM,https://store.akamai.steamstatic.com/public/sh...,English,2013.0,Jul,NaN,NaN,NaN,NaN
13406,Stop Sign VR Tools,Windows,Rp 48 999,Reality Inside Ltd.,Reality Inside Ltd.,Positive,"Indie,Action,Sports,Early Access,VR,Utilities,...","Action,Indie,Sports,Early Access",2048 MB RAM,NaN,English,2019.0,Dec,NaN,NaN,92.0,41.0
12300,Andarilho,Windows,Rp 39 999,SC Jogos,SC Jogos,Mixed,"Survival,Simulation,Crafting,Open World,Sandbo...","Action,Adventure,Casual,Indie,Simulation",4 GB RAM,NaN,English,2020.0,Nov,68.0,918.0,63.0,11.0


In [20]:
# exctract suported language

df_games_extract_3 = df_games_extract_2.copy()
df_games_extract_3['supported_language'] = df_games_extract_3['language'].str.split(',').apply(lambda x: len(x) if type(x) == list else x)
#df_games_extract_3['english_language'] = df_games_extract_3['language'].str.lower().str.contains('english')
df_games_extract_3 = df_games_extract_3.drop('language',axis=1)
df_games_extract_3.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,tags,genre,ram,rating,year,month,user_rating_all,total_review_all,user_rating_recent,total_review_recent,supported_language
17926,ZHP: Unlosing Ranger vs. Darkdeath Evilman Dig...,Windows,Rp 206 997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
113874,Sole Iron Tail,"Windows,Linux",Rp 39 999,Sunset Dreamland,Sunset Dreamland,Positive,"Fast-Paced,Runner,Collectathon,Platformer,Acti...","Action,Adventure,Casual,Indie,Simulation",4 GB RAM,NaN,2021.0,Nov,NaN,NaN,100.0,11.0,1.0
86852,Runster Demo,Windows,Free Demo,Runnex Technology,Runnex Technology,Mostly Positive,"Sports,Free to Play,Action,Indie,Casual,Early ...","Action,Casual,Free to Play,Indie,Sports",4 GB RAM,https://store.akamai.steamstatic.com/public/sh...,2019.0,Aug,NaN,NaN,78.0,234.0,2.0
88822,Rocksmith® 2014 Edition – Remastered – Manic S...,"Windows,Mac OS",Rp 59 999,Ubisoft - San Francisco,NaN,NaN,"Casual,Simulation","Casual,Simulation",2 GB RAM,https://store.akamai.steamstatic.com/public/sh...,2019.0,Jul,NaN,NaN,NaN,NaN,6.0
87805,American Truck Simulator Demo,"Windows,Mac OS,Linux",Free Demo,SCS Software,SCS Software,Overwhelmingly Positive,"Automobile Sim,Driving,Transportation,Open Wor...","Indie,Simulation",4 GB RAM,NaN,2016.0,Feb,NaN,NaN,96.0,79819.0,24.0


In [26]:
# extract the price 

df_games_price = df_games_extract_3.copy()
df_games_price['price'] = df_games_price['original_price'].str.lower().str.replace(r'\D','').astype(int,errors='ignore')
df_games_price.loc[df_games_price['price']=='','price'] = 0
df_games_price['price'] = df_games_price['price'].astype(float)
df_games_price = df_games_price.drop('original_price',axis=1)
df_games_price.sample(5)

C:\Users\nyash\AppData\Local\Temp\ipykernel_6196\2770343087.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_price['price'] = df_price['original_price'].str.lower().str.replace(r'\D','').astype(int,errors='ignore')


,title,platforms,developer,publisher,overall_reviews,tags,genre,ram,rating,year,month,user_rating_all,total_review_all,user_rating_recent,total_review_recent,supported_language,price
8085,Impostor Factory - The Bestest Dancers Comic,"Windows,Mac OS,Linux",Freebird Games,NaN,Very Positive,"Adventure,Indie,Comic Book,RPG","Adventure,Indie",1 GB RAM,NaN,2021.0,Sep,NaN,NaN,100.0,80.0,12.0,17499.0
85208,Arena of Shaelo,"Windows,Linux",Royal Donut,Royal Donut,6 user reviews,"Casual,Strategy,Indie,Card Game,Local Co-Op,Pi...","Casual,Indie,Strategy",2 GB RAM,NaN,2019.0,Feb,NaN,NaN,NaN,NaN,1.0,0.0
77230,X-Plane 11 - Add-on: MSK Productions - New Isl...,Windows,NaN,NaN,NaN,"Casual,Simulation,Relaxing,Exploration,Physics...","Casual,Simulation",NaN,NaN,2021.0,Jul,NaN,NaN,NaN,NaN,1.0,241744.0
82050,Meawja,"Windows,Linux",Galope,Galope,NaN,"Ninja,Action-Adventure,Platformer,Pixel Graphi...","Action,Adventure,Indie",2 GB RAM,NaN,NaN,None,NaN,NaN,NaN,NaN,4.0,NaN
10757,RICO,Windows,Ground Shatter,Rising Star Games,Mixed,"Action,Indie,Action Roguelike,Violent,FPS,Loca...","Action,Indie",4 GB RAM,NaN,2019.0,Mar,NaN,NaN,65.0,285.0,7.0,108999.0


In [30]:
# extract age and pegi rating

df_games_pegi = df_games_price.copy()
df_games_pegi['pegi_rated'] = np.where(df_games_pegi['rating']=='Rating for: PEGI', 1, 0)
df_games_pegi['age_rating'] = df_games_pegi['rating'].str.replace(r'\D','')
df_games_pegi = df_games_pegi.drop('rating',axis=1)
df_games_pegi.sample(5)

C:\Users\nyash\AppData\Local\Temp\ipykernel_6196\2760635418.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_pegi['age_rating'] = df_pegi['rating'].str.replace(r'\D','')


,title,platforms,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,total_review_all,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating
90554,Spaceland: Cooperative,Windows,Tortuga Team,Tortuga Team,NaN,"Strategy,Adventure,RPG,Indie,Turn-Based,Co-op,...","Adventure,Indie,RPG,Strategy",2 GB RAM,2021.0,Apr,NaN,NaN,NaN,NaN,15.0,0.0,0,NaN
91671,Pine Lily Village,"Windows,Mac OS,Linux",Laborious Lark Studios,Laborious Lark Studios,NaN,"Farming Sim,Life Sim,RPG,Pixel Graphics,Agricu...","Adventure,Casual,Indie,RPG,Simulation",2 GB RAM,NaN,None,NaN,NaN,NaN,NaN,1.0,NaN,0,NaN
11560,Babble Royale,"Windows,Mac OS",Everybody House Games,Everybody House Games,Mostly Positive,"Early Access,Action,Casual,Battle Royale,Word ...","Action,Casual,Free to Play,Indie,Early Access",3 GB RAM,2021.0,Dec,82.0,818.0,76.0,13.0,1.0,0.0,0,NaN
21119,Ukrainian Indie Games Bundle,Windows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,217491.0,0,NaN
79463,On A Roll 3D - Soundtrack,NaN,Battenberg Software,Battenberg Software,8 user reviews,"Action,Adventure,Indie,Casual,Soundtrack,Great...","Action,Adventure,Casual,Indie",NaN,2016.0,Sep,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN


In [33]:
# extract "Windows, Mac, Linux and VR" and create new columns

df_games_platform = df_games_pegi.copy()
df_games_platform['windows'] = np.where(df_games_platform['platforms'].str.contains('Windows',na=False), 1, 0)
df_games_platform['mac'] = np.where(df_games_platform['platforms'].str.contains('Mac OS',na=False), 1, 0)
df_games_platform['linux'] = np.where(df_games_platform['platforms'].str.contains('Linux',na=False), 1, 0)
df_games_platform['VR'] = np.where(df_games_platform['platforms'].str.contains('VR Supported',na=False), 1, 0)
df_games_platform = df_games_platform.drop('platforms',axis=1)
df_games_platform.sample(5)

,title,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,...,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating,windows,mac,linux,VR
111245,The Complete Halfquake Trilogy Soundtrack,Philipp Lehner,NaN,Positive,"Action,Adventure,Free to Play","Action,Adventure,Free to Play",256 MB RAM,2018.0,Oct,NaN,...,100.0,18.0,NaN,0.0,0,NaN,0,0,0,0
24125,Metro Exodus Expansion Pass,4A Games,Deep Silver,Very Positive,"Action,FPS,Singleplayer",Action,8 GB RAM,2019.0,May,NaN,...,87.0,435.0,9.0,161957.0,0,18,1,0,0,0
108089,Music Man,"Tero Lunkka,Valkeala Software,NTStudios",Valkeala Software,3 user reviews,"Adventure,Action,Casual,Platformer,Walking Sim...","Action,Adventure,Indie",4 GB RAM,2022.0,Jun,NaN,...,NaN,NaN,1.0,78999.0,0,NaN,1,0,0,0
51435,Gravity Vector,POLIEXA Entertainment,POLIEXA Entertainment,5 user reviews,"Action,Casual,Simulation,Space,Funny","Action,Casual,Simulation",4 GB RAM,2019.0,Jun,NaN,...,NaN,NaN,1.0,8499.0,0,NaN,1,0,0,0
82538,Cutthroat,Ocumens LLC,Ocumens LLC,2 user reviews,"Action,Indie,Casual,4 Player Local,Local Multi...","Action,Casual,Indie",512 MB RAM,2017.0,Nov,NaN,...,NaN,NaN,1.0,95999.0,0,NaN,1,1,1,0


In [34]:
# Remapping Value of The Column RAM

df_games_ram = df_games_platform.copy()
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('9 gb', na=False),'ram'] = '9 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('8 gb|8000 mb|８ gb|8gb|8.0 gb|8\+ gb', na=False),'ram'] = '8 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('7 gb', na=False),'ram'] = '7 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('6 gb|6000 mb|６ gb', na=False),'ram'] = '6 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('5 gb|5000 mb', na=False),'ram'] = '5 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('4 gb|4096 mb|4000 mb|4gb|４ gb|4.00 gb|4g|4 ram gb', na=False),'ram'] = '4 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('3 gb|3000 mb|3 go mb|3gb', na=False),'ram'] = '3 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('2 gb|2048|2000 mb|2gb|1536 mb|2.0 gb|2000 gb|２ gb|1.5gb|2.0gb|2g', na=False),'ram'] = '2 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('1 gb|1024|1000 mb|1gb|800 mb|750 mb|1280 mb|700 mb|900 mb|１ gb|1\+', na=False),'ram'] = '1 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('512|500 mb|400 mb|512mb|600 mb|597 mb|320 mb|420 mb|500mb|349 mb', na=False),'ram'] = '512 MB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('256 gb|200 mb|300 mb|250 mb|256mb|220 mb|256 ram', na=False),'ram'] = '256 MB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('16.0 gb', na=False),'ram'] = '16 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('12.00 gb', na=False),'ram'] = '12 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('10 gb', na=False),'ram'] = '10 GB'
df_games_ram.loc[df_games_ram['ram'].str.lower().str.contains('128 mb|64m|70 mb|100 mb|150 mb|16 mb|8 mb|2 mb|1 mb|4 mb|50 mb|3 mb|6 mb|10 mb|128mb|80 mb|4 ram mb|30 mb|40 mb|25 mb|5 mb|120 mb|64 mb|90 mb|60 mb|95 mb|65 mb|64mb|16mb|100以上|32mb|97 mb|20 mb|99 mb|4mb|69 mb|59 mb', na=False),'ram'] = '<128 MB'
df_games_ram.loc[~df_games_ram['ram'].str.lower().str.contains('16 gb|12 gb|10 gb|9 gb|8 gb|7 gb|6 gb|5 gb|4 gb|3 gb|2 gb|1 gb|512 mb|256 mb|<128 mb', na=False),'ram'] = 'Unknown'
df_games_ram['ram'].value_counts()

4 GB       27093
2 GB       23318
Unknown    22392
1 GB       15609
<128 MB    15445
8 GB       12899
6 GB        3784
3 GB        1103
5 GB         200
10 GB         41
9 GB          12
7 GB           6
12 GB          2
Name: ram, dtype: int64

In [35]:
df_games_ram.sample(10)

,title,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,...,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating,windows,mac,linux,VR
51602,DRAW CHILLY - Soundtrack,AZAMATIKA,AZAMATIKA,3 user reviews,"Action,Indie,Casual","Action,Casual,Indie",Unknown,2019.0,Nov,NaN,...,NaN,NaN,NaN,17499.0,0,NaN,0,0,0,0
46443,RPG Maker MV - Medieval Warfare Music Pack,"Murray Atkinson,VGM Library",Degica,2 user reviews,"Design & Illustration,Web Publishing","Design & Illustration,Web Publishing",<128 MB,2017.0,Jun,NaN,...,NaN,NaN,12.0,169999.0,0,NaN,1,1,1,0
24041,Vampire: The Masquerade - Heartless Lullaby,Entalto Studios,Entalto Studios,Very Positive,"RPG,Vampire,Story Rich,Choices Matter,Isometri...","Adventure,Free to Play,Indie,RPG",8 GB,2022.0,Apr,82.0,...,86.0,22.0,2.0,0.0,0,18,1,0,0,0
42747,Joker Poker - Video Poker,eSolutions,eSolutions Nordic AB,2 user reviews,"Simulation,Gambling,Card Game,Casual,Arcade,2D...","Casual,Indie,Simulation",4 GB,2022.0,May,NaN,...,NaN,NaN,1.0,39999.0,0,18,1,1,1,0
77995,Happy Empire 2 – The Lost Relic,AEL Entertainment,AEL Entertainment,NaN,"Strategy,Indie,Casual,Adventure","Adventure,Casual,Indie,Strategy",<128 MB,2018.0,Jul,NaN,...,NaN,NaN,1.0,NaN,0,NaN,1,0,0,0
74284,Aurora's Journey and the Pitiful Lackey,the Not so Great team,the Not so Great team,NaN,"Adventure,Side Scroller,Shoot 'Em Up,Platforme...","Action,Adventure,Indie",Unknown,2022.0,None,NaN,...,NaN,NaN,2.0,NaN,0,12,1,0,1,0
70766,Brocula,Prateek Jadhwani,Prateek Jadhwani,NaN,"RPG,Strategy,Simulation,Exploration,Management...","Adventure,Indie,RPG,Strategy",2 GB,NaN,None,NaN,...,NaN,NaN,7.0,NaN,0,NaN,1,1,0,0
84455,Don't Feed The Slimes!,ChillFun,ChillFun,NaN,"Strategy,Adventure,Indie,Casual,Early Access,P...","Adventure,Casual,Indie,Strategy,Early Access",1 GB,2018.0,Jun,NaN,...,NaN,NaN,1.0,32999.0,0,NaN,1,0,0,0
71196,Vacuum Pilot Demo,"VacuumDev,Jackerino",VacuumDev,Positive,"Time Attack,Multiplayer,Level Editor,Fast-Pace...","Racing,Early Access",2 GB,2021.0,Aug,NaN,...,100.0,13.0,9.0,0.0,0,NaN,1,0,0,0
45057,The Final Hour,ASGames,Alwin Sänger,5 user reviews,"Early Access,Horror,Atmospheric,Realistic,Hist...","Action,Adventure,Early Access",8 GB,2022.0,Mar,NaN,...,NaN,NaN,2.0,95999.0,0,NaN,1,0,0,0


In [36]:
# remap overall_reviews

df_games_or = df_games_ram.copy()
df_games_or.loc[~df_games_or['overall_reviews'].str.contains('Positive|Negative|Very|Overwhelmingly|Mixed|Mostly', na=False),'overall_reviews'] = '<10 reviews'
df_games_or['overall_reviews'].value_counts()

<10 reviews                76707
Very Positive              12821
Mixed                      10959
Positive                   10931
Mostly Positive             7928
Mostly Negative             1718
Overwhelmingly Positive      506
Negative                     256
Very Negative                 69
Overwhelmingly Negative        9
Name: overall_reviews, dtype: int64

In [38]:
# remap and reformat tags

def to_1D(series): #list feature function
 return pd.Series([x for _list in series for x in _list])

df_games_tags = df_games_or.copy()
df_games_tags.loc[df_games_tags['tags'].isna()==True,'tags'] = 'No Tags'
df_games_tags['tags'] = df_games_tags['tags'].str.split(',')
#df_games_tags['tags'] = df_games_tags['tags'].replace(',', '","')
#df_games_tags['tags'] = '["' + df_games_tags['tags'] + '"]'

In [39]:
to_1D(df_games_tags['tags']).value_counts().head(50)

Indie                 64062
Action                48498
Adventure             40897
Casual                40604
Singleplayer          40495
Simulation            25750
Strategy              24489
RPG                   23598
2D                    21655
Atmospheric           14959
Puzzle                13578
Pixel Graphics        12985
3D                    12793
Story Rich            12445
Fantasy               11968
Colorful              11408
Early Access          10715
Multiplayer           10550
Exploration           10539
No Tags                9884
Cute                   9607
First-Person           9602
Arcade                 9059
Free to Play           9001
Sci-fi                 8388
Shooter                8362
Funny                  8356
Platformer             8147
Retro                  7850
Action-Adventure       7634
Family Friendly        7577
Horror                 7309
Anime                  7228
Relaxing               7133
Violent                6987
Difficult           

In [41]:
# remap and refomart genre

df_games_genre = df_games_tags.copy()
df_games_genre.loc[df_games_genre['genre'].isna()==True,'genre'] = 'No Genre'
df_games_genre['genre'] = df_games_genre['genre'].str.split(',')
#df_games_tags['tags'] = df_games_tags['tags'].replace(',', '","')
#df_games_tags['tags'] = '["' + df_games_tags['tags'] + '"]'
df_games_genre.sample(5)

,title,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,...,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating,windows,mac,linux,VR
116592,Gobby McGobblenutz Presents: The Art of the Da...,Randumb Studios,Randumb Studios,Very Positive,"[Simulation, Choices Matter, Choose Your Own A...","[Casual, Indie, RPG, Simulation]",Unknown,2021.0,Nov,NaN,...,92.0,119.0,1.0,17499.0,0,NaN,1,0,0,0
115264,Sherwood Extreme,CAGE Studios,CAGE Studios,Mostly Positive,"[Early Access, Third-Person Shooter, Archery, ...","[Action, Adventure, Casual, Free to Play, Indi...",4 GB,2021.0,May,80.0,...,73.0,15.0,4.0,0.0,0,NaN,1,0,0,0
11774,Marlow Briggs and the Mask of Death,Zootfly,505 Games,Very Positive,"[Action, Spectacle fighter, Hack and Slash, Ad...","[Action, Adventure]",2 GB,2013.0,Sep,NaN,...,91.0,1488.0,5.0,45999.0,0,16,1,0,0,0
5153,Almighty: Kill Your Gods,RUNWILD Entertainment,Versus Evil,Mostly Positive,"[Early Access, Looter Shooter, Hack and Slash,...","[Action, RPG, Early Access]",8 GB,2021.0,May,NaN,...,70.0,270.0,1.0,175000.0,0,16,1,0,0,0
170,Cuphead,Studio MDHR Entertainment Inc.,Studio MDHR Entertainment Inc.,Overwhelmingly Positive,"[Difficult, Cartoon, Platformer, Co-op, Great ...","[Action, Indie]",3 GB,2017.0,Sep,96.0,...,96.0,1678.0,12.0,135999.0,0,7,1,1,0,0


In [42]:
to_1D(df_games_genre['genre']).value_counts()

Indie                    71365
Action                   47056
Adventure                38306
Casual                   37885
Simulation               24781
Strategy                 23237
RPG                      22660
No Genre                 10570
Early Access              8972
Free to Play              8557
Sports                    5055
Racing                    4322
Massively Multiplayer     3360
Design & Illustration     2257
Web Publishing            1817
Utilities                 1325
Animation & Modeling       707
Education                  631
Video Production           465
Software Training          442
Game Development           428
Audio Production           352
Photo Editing              303
Accounting                  23
Movie                        3
Documentary                  1
Episodic                     1
Short                        1
Tutorial                     1
360 Video                    1
dtype: int64

In [44]:
# reformat and remap developer

df_games_dev = df_games_genre.copy()
df_games_dev.loc[df_games_dev['developer'].isna()==True,'developer'] = 'Unknown'
df_games_dev['developer'] = df_games_dev['developer'].str.split(',')
#df_games_tags['tags'] = df_games_tags['tags'].replace(',', '","')
#df_games_tags['tags'] = '["' + df_games_tags['tags'] + '"]'
df_games_dev.sample(5)

,title,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,...,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating,windows,mac,linux,VR
31094,Independent Space Alliance,[Unknown],NaN,<10 reviews,[No Tags],[No Genre],Unknown,NaN,None,NaN,...,NaN,NaN,NaN,558800.0,0,16,1,0,0,0
99022,Fantasy Grounds - D&D Adventurers League 10-03...,"[SmiteWorks USA, LLC]",NaN,<10 reviews,"[Strategy, RPG, Indie]","[Indie, RPG, Strategy]",1 GB,2021.0,Oct,NaN,...,NaN,NaN,1.0,39999.0,0,NaN,1,1,0,0
46919,The Shepherd,[MW Hashem],MeemWow Games,<10 reviews,"[RPG, Turn-Based Combat, Exploration, Singlepl...","[Indie, RPG, Simulation]",1 GB,2022.0,Mar,NaN,...,NaN,NaN,2.0,78999.0,0,NaN,1,1,0,0
23146,KF2 - Digital Deluxe Edition DLC,[Tripwire Interactive],Tripwire Interactive,Very Positive,[Action],[Action],Unknown,2016.0,Nov,NaN,...,93.0,66.0,1.0,70000.0,0,18,1,0,1,0
85075,La Corda d'Oro Octave/金色のコルダ オクターヴ - ホワイトデー・響也,"[KOEI TECMO GAMES CO., LTD.]","KOEI TECMO GAMES CO., LTD.",<10 reviews,[Adventure],[Adventure],2 GB,2019.0,Mar,NaN,...,NaN,NaN,2.0,53000.0,0,NaN,1,0,0,0


In [45]:
to_1D(df_games_dev['developer']).value_counts().head(50)

Unknown                           7983
 LLC                              2955
SmiteWorks USA                    2375
TigerQiuQiu                       2132
 LTD.                             2049
KOEI TECMO GAMES CO.              1857
Ubisoft - San Francisco           1704
 Ltd.                              905
 Inc.                              800
CAPCOM Co.                         534
N3V Games                          446
Dovetail Games                     400
Feral Interactive (Mac)            327
Choice of Games                    296
Nihon Falcom                       287
 Inc                               285
Milestone S.r.l.                   265
Feral Interactive (Linux)          258
Square Enix                        255
Idea Factory                       232
Inc.                               229
Paradox Development Studio         217
Harmonix Music Systems             215
Tamsoft                            210
Arc System Works                   198
Rebellion                

In [46]:
#reformat and remap publisher

df_games_pub = df_games_dev.copy()
df_games_pub.loc[df_games_pub['publisher'].isna()==True,'publisher'] = 'Unknown'
df_games_pub['publisher'] = df_games_pub['publisher'].str.split(',')
df_games_pub.sample(5)

,title,developer,publisher,overall_reviews,tags,genre,ram,year,month,user_rating_all,...,user_rating_recent,total_review_recent,supported_language,price,pegi_rated,age_rating,windows,mac,linux,VR
29081,Kings Town,[Leturs Labs],[Leturs Labs],Mixed,"[Early Access, Strategy, Building, RTS, Surviv...","[Casual, Indie, RPG, Strategy, Early Access]",4 GB,2020.0,Oct,NaN,...,66.0,21.0,1.0,69999.0,0,NaN,1,0,0,0
97166,Swung,"[Thomas Huijser, Rens Koning, Jesse van Rijn]",[Extra Nice],<10 reviews,"[Adventure, Action, Swordplay, Medieval, 2D Fi...","[Action, Adventure, Casual, Indie]",4 GB,NaN,None,NaN,...,NaN,NaN,1.0,NaN,0,NaN,1,0,0,0
88293,Oddworld: Soulstorm Enhanced Edition Demo,[Oddworld Inhabitants],[Oddworld Inhabitants],<10 reviews,"[Action, Adventure, Action-Adventure, Platform...","[Action, Adventure, Indie]",Unknown,2022.0,Jun,NaN,...,NaN,NaN,12.0,0.0,0,NaN,1,0,0,0
54378,Primal Carnage - Pilot Commando DLC,[Lukewarm Media],[Reverb Publishing],<10 reviews,"[Action, Indie, Strategy]","[Action, Strategy, Indie]",2 GB,2013.0,Jan,NaN,...,NaN,NaN,1.0,9299.0,0,NaN,1,0,0,0
39811,Book of Legends,[Gogii],[iWin],<10 reviews,"[Adventure, Hidden Object]",[Adventure],<128 MB,2010.0,Jul,NaN,...,NaN,NaN,1.0,89999.0,0,NaN,1,0,0,0


In [48]:
to_1D(df_games_pub['publisher']).value_counts().head(50)

Unknown                       18014
TigerQiuQiu                    2132
 LTD.                          1692
Degica                         1690
KOEI TECMO GAMES CO.           1552
 Inc.                          1224
 Ltd.                           834
 LLC                            608
BANDAI NAMCO Entertainment      604
Dovetail Games - Trains         601
SEGA                            538
CAPCOM Co.                      523
Square Enix                     490
N3V Games                       487
Paradox Interactive             484
Ubisoft                         441
Big Fish Games                  405
Electronic Arts                 373
XSEED Games                     367
Marvelous USA                   360
THQ Nordic                      332
D3 PUBLISHER                    317
PlayWay S.A.                    303
Feral Interactive (Mac)         299
Choice of Games                 294
2K                              274
Milestone S.r.l.                258
Slitherine Ltd.             